<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/insurance_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain langchain_huggingface langchain_community  gradio transformers elasticsearch gradio_calendar

In [2]:
import os
from getpass import getpass
import requests
import os
from langchain import PromptTemplate, LLMChain, HuggingFaceHub
from huggingface_hub import InferenceClient
from elasticsearch import Elasticsearch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import json
import gradio as gr
from datetime import datetime

## gradio

In [3]:
## LLM credentials
username_llm = "manageai2024"
passwords_llm  = "ManageAI@2024"

# Elasticsearch credentials
user = "natthaphol.po"
passwords = "cnEM5CeFrG"

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
es = Elasticsearch(
    hosts=[{'host': 'elk.manageai.co.th', 'port': 443, 'scheme': 'https'}],
    basic_auth=(user, passwords)
)
3
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def rag_encode(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings.squeeze().tolist()

def search_by_icd10(query_icd10, index_name):
    search_query = {
        "size": 1000,
        "_source": [
            "simb_billing_code", "local_billing_name", "item_id", "item_name",
            "average_initial_price", "order_count", "icd10", "range"
        ],
        "query": {
            "match": {
                "icd10": query_icd10  # ค้นหาจากคอลัมน์ icd10
            }
        }
    }

    try:
        response = es.search(index=index_name, body=search_query, timeout='120s')
        results = []
        for hit in response['hits']['hits']:
            result = {
                "_index": hit["_index"],
                "_id": hit["_id"],
                "_source": hit["_source"]
            }
            results.append(result)

        if results:
            source_data = [hit["_source"] for hit in results]
            df = pd.DataFrame(source_data)
            return df
        else:
            print("No results found.")
            return None

    except Exception as e:
        print(f"Error performing search: {e}")
        return None


In [4]:
# Store claimed records
claimed_records = set()

def handle_form(data_json):
    try:
        data = json.loads(data_json)

        # Validate required fields
        required_fields = [
            "policy_start_date", "policy_end_date", "visit_date",
            "patient_id", "diagnosis", "total_net", "remaining_opd_year", "order_items"
        ]
        for field in required_fields:
            if field not in data:
                raise ValueError(f"Missing required field: {field}")

        # Check for duplicate claims
        claim_tuple = tuple(sorted(data.items()))  # Create a unique tuple for the claim
        if claim_tuple in claimed_records:
            result = {
                "text": "Decline ❌",
                "entities": [{"start": 0, "end": 7, "label": "Decline"}]
            }
            reason = "Duplicate claim detected: The claim data has already been submitted."
            return result, reason, []

        # Add claim to records to prevent duplicates
        claimed_records.add(claim_tuple)

        # Extract and validate key information
        policy_start_date = data["policy_start_date"]
        policy_end_date = data["policy_end_date"]
        visit_date = data["visit_date"]
        diagnosis = data["diagnosis"]
        total_net = float(data["total_net"])
        remaining_opd_year = float(data["remaining_opd_year"])

        # Parse `order_items` into a list of dictionaries
        try:
            order_items = eval(data["order_items"])  # Convert string representation of list to actual list
        except Exception as e:
            return {"text": "Error ⚠️", "entities": []}, f"Failed to parse `order_items`: {e}", []

        # Extract `item_id` values from order_items
        items_used = {item['item_id'] for item in order_items if 'item_id' in item}

        # Convert dates for comparison
        policy_start = datetime.strptime(policy_start_date, '%Y-%m-%d')
        policy_end = datetime.strptime(policy_end_date, '%Y-%m-%d')
        visit = datetime.strptime(visit_date, '%Y-%m-%d')

        # Extract ICD10 code from diagnosis
        icd10_code = diagnosis.split(' ')[0] if diagnosis else None
        if not icd10_code:
            return {"text": "Error ⚠️", "entities": []}, "ICD10 code is missing or invalid in the diagnosis.", []

        # Fetch item data associated with the ICD10
        item_data = search_by_icd10(icd10_code, "internal-manageai-icd10-order")
        if item_data is None or item_data.empty:
            return {"text": "Error ⚠️", "entities": []}, f"No item data found for ICD10: {icd10_code}", []

        # Compare patient items with ICD10-associated items
        icd10_items = set(item_data["item_id"].tolist())
        unmatched_items = items_used - icd10_items

        if unmatched_items:
            # Convert unmatched items to a list of lists (for Dataframe)
            unmatched_list = [
                [
                    item_data.loc[item_data["item_id"] == item, "simb_billing_code"].values[0],  # simb_billing_code
                    item_data.loc[item_data["item_id"] == item, "local_billing_name"].values[0],  # local_billing_name
                    item,  # item_id
                    item_data.loc[item_data["item_id"] == item, "item_name"].values[0],  # item_name
                    item_data.loc[item_data["item_id"] == item, "average_initial_price"].values[0],  # average_initial_price
                    1,  # order_count (for simplicity, assuming 1 order count)
                    icd10_code,  # icd10
                    "N/A"  # range (can be adjusted based on your data)
                ]
                for item in unmatched_items
            ]
            result = {"text": "Decline ❌", "entities": [{"start": 0, "end": 7, "label": "Decline"}]}
            reason = f"The following items used are not associated with ICD10 {icd10_code}: {', '.join(unmatched_items)}"
            return result, reason, unmatched_list

        # Check for waiting period diseases
        wait_disease = ["K30", "K297", "I495"]
        if icd10_code in wait_disease:
            result = {"text": "Decline ❌", "entities": [{"start": 0, "end": 7, "label": "Decline"}]}
            reason = f'เนื่องจาก {diagnosis} เป็นโรคที่มีระยะเวลาการรอคอย 180 วัน'

        # Check visit date within policy coverage period
        elif visit < policy_start or visit > policy_end:
            result = {"text": "Decline ❌", "entities": [{"start": 0, "end": 7, "label": "Decline"}]}
            reason = f"Visit date {visit_date} is outside the policy coverage period ({policy_start_date} to {policy_end_date})."

        # Check if total bill exceeds remaining coverage
        elif total_net > remaining_opd_year:
            result = {"text": "Decline ❌", "entities": [{"start": 0, "end": 7, "label": "Decline"}]}
            reason = f"Total bill amount {total_net} exceeds the remaining OPD coverage ({remaining_opd_year})."

        else:
            result = {"text": "Approve ✅", "entities": [{"start": 0, "end": 7, "label": "Approve"}]}
            reason = "All items used are appropriate for the diagnosed condition, visit date is within the policy coverage period, and total bill amount is within the remaining coverage."

        return result, reason, []

    except ValueError as e:
        result = {"text": "Error ⚠️", "entities": []}
        reason = f"Invalid input or date format. Please use YYYY-MM-DD. {str(e)}"
        return result, reason, []

    except Exception as e:
        result = {"text": "Error ⚠️", "entities": []}
        reason = f"An unexpected error occurred: {str(e)}"
        return result, reason, []

In [5]:
custom_css = """
.red-border {
    border: 2px solid red !important;
    border-radius: 5px;
    padding: 5px;
}
"""

### UI
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("## SmartClaim ฝั่งประกัน 🛡️")
    data = gr.Textbox(label="ข้อมูลผู้ป่วย", placeholder="กรอกข้อมูลผู้ป่วยในรูปแบบ JSON แล้ว Enter หรือ Submit")
    submit_button = gr.Button("Submit")
    gr.Markdown("### ไอเท็มที่ผู้ป่วยโรคเดียวกันกับคนไข้จะไม่สั่ง")
    item_table = gr.Dataframe(headers=[
        "simb_billing_code", "local_billing_name", "item_id", "item_name",
        "average_initial_price", "order_count", "icd10", "range"
    ], interactive=False)

    gr.Markdown("### Insurance Claim Decision System")
    with gr.Row(elem_classes="red-border"):
        result_output = gr.HighlightedText(label="Result", interactive=False)
        reason_output = gr.Textbox(label="Reason", interactive=False)

        data.submit(
            fn=handle_form,
            inputs=[data],
            outputs=[result_output, reason_output, item_table]
        )
        submit_button.click(
            fn=handle_form,
            inputs=[data],
            outputs=[result_output, reason_output, item_table]
        )

In [ ]:
demo.launch(debug=True)

## **decline case**

In [58]:
data = pd.read_csv('/content/drive/MyDrive/AIEngineer/fraud/opd_discharge_August_2024.csv')

In [64]:
data[data['cid']=='1710500385213'][['created_at','cid','patient','claim_status','response','is_follow_up','billing','order_item','total_bill_amount','insurance','diagnosis','chief_complaint']]

,created_at,cid,patient,claim_status,response,is_follow_up,billing,order_item,total_bill_amount,insurance,diagnosis,chief_complaint
5474,2024-08-27 14:15:19.160 +0700,1710500385213,"{""hn"": ""01-24-031234"", ""dob"": ""1999-02-21"", ""g...",Submit,"{""detail"": [], ""claim_no"": ""C029226914"", ""insu...",N,"[{""billing_initial"": ""1090.00"", ""billing_disco...","[{""initial"": ""90"", ""item_id"": ""16422"", ""discou...",2341.0,aia,"[{""icd10"": ""U071"", ""dx_name"": ""COVID-19, virus...",ไข้ 1 วัน


In [65]:
## item ที่สั่งไม่ตรงกับโรคที่เป็น
## จริงๆคนนี้เป็น J069 การติดเชื้อทางเดินหายใจส่วนบนเฉียบพลัน ไม่ระบุรายละเอียด (Acute upper respiratory infection, unspecified [ URI ])
## เทสเปลี่ยนเป็น U071 โควิด-19 ระบุเชื้อไวรัส (COVID-19, virus identified)
{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "คณิศร",
    "last_name": "ทองดีบุตร",
    "patient_id": "6609393",
    "birth_date": "1997-12-22",
    "visit_date": "2024-08-12",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "policy_start_date": "2023-01-01",
    "policy_end_date": "2025-01-01",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "U071 โควิด-19 ระบุเชื้อไวรัส (COVID-19, virus identified)",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "7000",
    "total_net": "1491.0",
    "final_result": "6657.75",
    "recommendation_adjustment": "+ 5166.75"
}

In [ ]:
##  ไม่ครอบคลุมจาก Uncoverage Period
##  policy_start_date, policy_end_date, visit_date
{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "คณิศร",
    "last_name": "ทองดีบุตร",
    "patient_id": "6609393",
    "birth_date": "1997-12-22",
    "visit_date": "2024-08-13",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "policy_start_date": "2023-01-01",
    "policy_end_date": "2024-01-01",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "J069 การติดเชื้อทางเดินหายใจส่วนบนเฉียบพลัน ไม่ระบุรายละเอียด (Acute upper respiratory infection, unspecified [ URI ])",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "7000",
    "total_net": "1491.0",
    "final_result": "6657.75",
    "recommendation_adjustment": "+ 5166.75"
}

In [ ]:
##  โรคที่มีระยะเวลารอคอย 180 วัน
##  icd_10 from diagnosis  ["K30", "K297", "I495"]

{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "aaa",
    "last_name": "bbb",
    "patient_id": "147852369",
    "birth_date": "1997-12-22",
    "visit_date": "2024-08-13",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "policy_start_date": "2024-01-01",
    "policy_end_date": "2025-01-01",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "K30 อาหารไม่ย่อย (Functional Dyspepsia)",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "7000",
    "total_net": "1491.0",
    "final_result": "6657.75",
    "recommendation_adjustment": "+ 5166.75"
}

{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "aaa",
    "last_name": "bbb",
    "patient_id": "147852369",
    "birth_date": "1997-12-22",
    "visit_date": "2024-08-13",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "policy_start_date": "2024-01-01",
    "policy_end_date": "2025-01-01",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "K297 กระเพาะอาหารอักเสบ ไม่ระบุรายละเอียด (Gastritis, unspecified)",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "7000",
    "total_net": "1491.0",
    "final_result": "6657.75",
    "recommendation_adjustment": "+ 5166.75"
}

{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "aaa",
    "last_name": "bbb",
    "patient_id": "147852369",
    "birth_date": "1997-12-22",
    "visit_date": "2024-08-13",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "policy_start_date": "2024-01-01",
    "policy_end_date": "2025-01-01",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "I495 กลุ่มอาการซิคไซนัส (Sick sinus syndrome)",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "7000",
    "total_net": "1491.0",
    "final_result": "6657.75",
    "recommendation_adjustment": "+ 5166.75"
}


In [ ]:
# total_net มากผิดปกติ
{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "aaa",
    "last_name": "bbb",
    "patient_id": "147852369",
    "birth_date": "1997-12-22",
    "visit_date": "2024-08-13",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "policy_start_date": "2024-01-01",
    "policy_end_date": "2025-01-01",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "J069 การติดเชื้อทางเดินหายใจส่วนบนเฉียบพลัน ไม่ระบุรายละเอียด (Acute upper respiratory infection, unspecified [ URI ])",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "10000",
    "total_net": "1000000.0",
    "final_result": "6657.75",
    "recommendation_adjustment": "+ 5166.75"
}

In [ ]:
# duplicate claim: ต้องเหมือนกันทุกคอลัมน์
{
    "patient_id": "3300800217090",
    "visit_date": "2024-08-13",
    "policy_start_date": "2024-01-01",
    "policy_end_date": "2025-01-01",
    "diagnosis": "Z48",
    "total_net": "1000",
    "remaining_opd_year": "7000"
}

{
    "patient_id": "3300800217090",
    "visit_date": "2024-08-13",
    "policy_start_date": "2024-01-01",
    "policy_end_date": "2025-01-01",
    "diagnosis": "Z48",
    "total_net": "1000",
    "remaining_opd_year": "7000"
}